In [1]:
import tensorflow as tf
import numpy as np
from hourglass_tiny import HourglassModel
import configparser
import os
import pandas as pd
from keras.preprocessing import image
import scipy.io
from scipy.misc import imresize
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

C:\Users\44163\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using TensorFlow backend.


In [16]:
# get the names of the data
person_set = pd.read_table('VOC2010/ImageSets/Main/person_train.txt',header=None)
d_names = []
data_len = len(person_set)
for i in range(data_len):
    if person_set[0][i][12:14] == ' 1':
        name = person_set[0][i][0:11]+'.jpg'
        d_names.append(name)
data_tabel = pd.DataFrame(d_names,columns=['d_name'])
# get names of masks
m_names = []
data_len = len(person_set)
for i in range(data_len):
    if person_set[0][i][12:14] == ' 1':
        name = person_set[0][i][0:11]+'.mat'
        m_names.append(name)
m_names = pd.DataFrame(m_names,columns=['m_name'])
data_tabel = pd.concat([data_tabel,m_names],axis=1)
# get the address of the data
address = []
for i in range(len(data_tabel)):
    path = os.path.join('VOC2010','JPEGImages',data_tabel['d_name'][i])
    address.append(path)
path = pd.DataFrame(address,columns=['d_path'])
data_tabel = pd.concat([data_tabel,path],axis=1)
# get address of mask
address = []
for i in range(len(data_tabel)):
    path = os.path.join('VOC2010','trainval','Annotations_Part',data_tabel['m_name'][i])
    address.append(path)
path = pd.DataFrame(address,columns=['m_path'])
data_tabel = pd.concat([data_tabel,path],axis=1)
data_tabel.head()

,d_name,m_name,d_path,m_path
0,2008_000008.jpg,2008_000008.mat,VOC2010\JPEGImages\2008_000008.jpg,VOC2010\trainval\Annotations_Part\2008_000008.mat
1,2008_000023.jpg,2008_000023.mat,VOC2010\JPEGImages\2008_000023.jpg,VOC2010\trainval\Annotations_Part\2008_000023.mat
2,2008_000036.jpg,2008_000036.mat,VOC2010\JPEGImages\2008_000036.jpg,VOC2010\trainval\Annotations_Part\2008_000036.mat
3,2008_000041.jpg,2008_000041.mat,VOC2010\JPEGImages\2008_000041.jpg,VOC2010\trainval\Annotations_Part\2008_000041.mat
4,2008_000096.jpg,2008_000096.mat,VOC2010\JPEGImages\2008_000096.jpg,VOC2010\trainval\Annotations_Part\2008_000096.mat


In [ ]:
# load data
def load_image(path):
    img = image.load_img(path,target_size=(256,256))
    img = image.img_to_array(img)
    return img

In [17]:
label = scipy.io.loadmat(data_tabel['m_path'][0])

In [107]:
len(label['anno'][0][0][1][0][0][3][0][0])

2

In [106]:
label['anno'][0][0][1][0][0][3][0]

array([(array(['head'], dtype='<U4'), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)),
       (array(['lear'], dtype='<U4'), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)),
       (array(['leye'], dtype='<U4'), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)),
       (array(['lebrow'], dtype='<U6'), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 

In [48]:
plt.figure()
plt.imshow(test)
plt.show()